**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 3 - Open Science Platform - Project Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a project account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the project account;

6 - Queries the blockchain again to confirm the succesfull details setting.

Every account succesfully created is dumped to `datasets/projects.json` for later use and evidence collection.  

In [1]:
# from name_generator import left, right
from utilities import *
from project_id_generator import * # - Random generated synthetic project ID
from iroha_helper import *
from loguru import logger
from dump_to_json import ProjectAccount, update_or_append_project_metadata
from ipfs_functions import download_json_from_ipfs
from synthetic_project_json_ld import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [2]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.  

In [3]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-10 15:02:53,811 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-10 15:02:53.811 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a project account into the blockchain;

In [4]:
project_private_key = IrohaCrypto.private_key()
project_public_key = IrohaCrypto.derive_public_key(project_private_key).decode("utf-8")

project_id = generate_project_id()

logger.info(f"Creating project with name: {project_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_project_account(create_account_contract_address, project_id, DOMAIN, project_public_key)


2025-02-10 15:02:53.867 | INFO     | __main__:<module>:6 - Creating project with name: 91276
2025-02-10 15:02:53.869 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-10 15:02:53.889 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-10 15:02:53.890 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:02:53.892 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-10 15:02:53.893 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_project_account"
2025-02-10 15:02:53,918 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:02:53,920 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:02:56,933 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:02

3 - Queries the blockchain to confirm the succesful creation of the account;

In [5]:
#Query - GetAccountDetail

# - Get account info
project_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(project_account_hash)


2025-02-10 15:02:56.967 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-10 15:02:56,970 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:02:56,986 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:02:56,988 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:02:59,964 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:02:59,973 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:02:59,975 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '672dee04636928777ab6326b49dad9025fd1579b1479423a00e8fb551ccab4c7', 'timestamp': 1739199779975}
2025-02-10 15:02:59.976 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-10 15:02:59,980 - INFO - Appended data under '91276@test' in logs/account_data.json
2025-02-10 15:02:59.981 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-10 15:02:59.982

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [6]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-10 15:03:00,023 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-10 15:03:00.025 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the project account:

In [7]:
# Generate synthetic project metadata JSON (assumed to be done already)
project_metadata_json = generate_synthetic_entry_ld()
account_metadata_cid = upload_json_to_ipfs(project_metadata_json)  # Assuming this is done before calling the function

# Call the function to update or append a new entry to the JSON-LD file
updated_entry_count = update_or_append_project_metadata(
    project_id, 
    project_metadata_json, 
    account_metadata_cid
)


address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

# Setting account detail using the generated project metadata CID
hash = set_account_detail(
    address, 
    f"{project_id}@{DOMAIN}",  # Project ID as account
    "account_metadata_cid",    # The key we're setting
    account_metadata_cid       # The value (CID from IPFS)
)

2025-02-10 15:03:00.123 | SUCCESS  | ipfs_functions:upload_json_to_ipfs:42 - Success uploading JSON to IPFS. CID: QmQarcPxD5dB6WdLfRf7vv1etu9mwUFfZwRjRydeAytros
2025-02-10 15:03:00.125 | INFO     | dump_to_json:update_or_append_project_metadata:216 - Updated existing project entry for project ID: 91276@test
2025-02-10 15:03:00.127 | INFO     | dump_to_json:update_or_append_project_metadata:235 - Current total entries in 'logs/projects.json': 1


2025-02-10 15:03:00.129 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-10 15:03:00.148 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-10 15:03:00.149 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-10 15:03:00.150 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-10 15:03:00.152 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-02-10 15:03:00,166 - INFO - None
2025-02-10 15:03:00,168 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:03:02,994 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:03:03,003 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:03:03.005 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_rec

6 - Queries the blockchain to confirm the succesful details setting

In [8]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid, project_id = process_raw_data(details) #TO DO - From now

2025-02-10 15:03:03.048 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-10 15:03:03,050 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:03:03,065 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:03:03,067 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:03:06,030 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:03:06,039 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:03:06,040 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '148c3bd958dd287dbd6a2c96a749cea2af85725ca987d27347c86b1a8d14b440', 'timestamp': 1739199786040}
2025-02-10 15:03:06.042 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-10 15:03:06,045 - INFO - Appended data under '91276@test' in logs/account_data.json
2025-02-10 15:03:06.046 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-10 15:03:06.048

7 - Retrieves Project metadata (JSON-LD formatted) from IPFS

In [9]:
project_metadata = download_json_from_ipfs(account_metadata_cid)

2025-02-10 15:03:06.104 | SUCCESS  | ipfs_functions:download_json_from_ipfs:52 - Success downloading JSON from IPFS. Metadata: {'@context': {'dc': 'http://purl.org/dc/terms/', 'schema': 'http://schema.org/'}, '@type': 'schema:ResearchProject', 'dc:abstract': 'This paper analyzes how satellite imagery influences precision medicine, providing insights into how to maximize its scientific discovery.', 'schema:endDate': '2027-06-28', 'schema:funding': {'@type': 'schema:Organization', 'schema:name': 'European Union Horizon 2020'}, 'schema:keywords': ['satellite imagery', 'precision medicine', 'scientific discovery'], 'schema:location': {'@type': 'schema:Place', 'schema:name': 'Yellowstone National Park, USA'}, 'schema:name': 'Analyzing the Influence of satellite imagery on precision medicine', 'schema:startDate': '2019-07-10'}
